# Case study for sequential optimization approaches using multitask elastic net and neural network, with 10-fold cross validation

In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error,root_mean_squared_error
from sklearn.decomposition import PCA
from sklearn.linear_model import MultiTaskElasticNetCV
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset,random_split

import optuna
import random

import pickle

Import cell lists and features

In [ ]:
training_cells = pd.read_csv("../Data_preprocessing/training.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()
test_in_cells = pd.read_csv("../Data_preprocessing/test_in.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()
test_out_cells = pd.read_csv("../Data_preprocessing/test_out.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()

num_training_cells = len(training_cells)
num_test_in_cells = len(test_in_cells)
num_test_out_cells = len(test_out_cells)

X_train_PCA = np.loadtxt("Processed_input_output/X_train_PCA.csv",delimiter=",")
X_test_in_PCA = np.loadtxt("Processed_input_output/X_test_in_PCA.csv",delimiter=",")
X_test_out_PCA = np.loadtxt("Processed_input_output/X_test_out_PCA.csv",delimiter=",")

N_train = np.loadtxt("Processed_input_output/N_train.csv",delimiter=",")
N_test_in = np.loadtxt("Processed_input_output/N_test_in.csv",delimiter=",")
N_test_out = np.loadtxt("Processed_input_output/N_test_out.csv",delimiter=",")

Q_train = np.array([np.linspace(1,0.8,21)]*num_training_cells)
Q_test_in = np.array([np.linspace(1,0.8,21)]*num_test_in_cells)
Q_test_out = np.array([np.linspace(1,0.8,21)]*num_test_out_cells)

Scale and transform training labels (b values)

In [ ]:
y_train = np.loadtxt("Processed_input_output/b_train.csv",delimiter=",")
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(np.log(y_train))

In [20]:
with open('training_cells_CV.pkl','rb') as f:
    training_cells_CV = pickle.load(f)

with open('val_cells_CV.pkl','rb') as f:
    val_cells_CV = pickle.load(f)


# Multitask elastic net model

Train the cross-validated multitask elastic net model

In [62]:
# Use predefined training and validation splits
cv_eln = []
for i in range(10):
    cv_eln.append((training_cells_CV[i],val_cells_CV[i]))

model = MultiTaskElasticNetCV(l1_ratio=[.1, .2, .5, .7, .9, .95, .99, 1],cv=cv_eln,n_jobs=-1,random_state=42,max_iter=1000)
# model.fit(X_train_PCA_mask,y_train_scaled)
model.fit(X_train_PCA,y_train_scaled)

print(f"best alpha: {model.alpha_}, best l1_ratio: {model.l1_ratio_}")

best alpha: 0.6824577644748759, best l1_ratio: 0.1


Predict parameters using the trained elastic net model

In [59]:
y_train_pred_eln_scaled = model.predict(X_train_PCA)
y_train_pred_eln = np.exp(y_scaler.inverse_transform(y_train_pred_eln_scaled))

y_test_in_pred_eln_scaled = model.predict(X_test_in_PCA)
y_test_in_pred_eln = np.exp(y_scaler.inverse_transform(y_test_in_pred_eln_scaled))

y_test_out_pred_eln_scaled = model.predict(X_test_out_PCA)
y_test_out_pred_eln = np.exp(y_scaler.inverse_transform(y_test_out_pred_eln_scaled))

Save the predicted empirical parameters

In [ ]:
b_train_pred_eln_df = pd.DataFrame(y_train_pred_eln,index=training_cells)
b_test_in_pred_eln_df = pd.DataFrame(y_test_in_pred_eln,index=test_in_cells)
b_test_out_pred_eln_df = pd.DataFrame(y_test_out_pred_eln,index=test_out_cells)

# Save as csv files
b_train_pred_eln_df.to_csv("Empirical_parameter_results/b_train_seq_eln_V2.csv")
b_test_in_pred_eln_df.to_csv("Empirical_parameter_results/b_test_in_seq_eln_V2.csv")
b_test_out_pred_eln_df.to_csv("Empirical_parameter_results/b_test_out_seq_eln_V2.csv")

Construct the predicted trajectory

In [ ]:
a = np.loadtxt('../Empirical_model_fitting/Empirical_parameters_global_train_py.csv').item()

def empirical_model_curve_fit(N_eq,b1,b2,b3):
    # b1,b2,b3 = b
    return 1 - b1*N_eq**a - 1/(1+np.exp((b2-N_eq)/(b3)))

Q_train_pred = np.ones_like(Q_train)
Q_test_in_pred = np.ones_like(Q_test_in)
Q_test_out_pred = np.ones_like(Q_test_out)

train_RMSE = []
test_in_RMSE = []
test_out_RMSE = []

train_MAE = []
test_in_MAE = []
test_out_MAE = []

for iii,b in enumerate(y_train_pred_eln):
    Q_train_pred[iii,:] = empirical_model_curve_fit(np.abs(N_train[iii]),b[0],b[1],b[2])
    train_RMSE.append(root_mean_squared_error(Q_train[iii],Q_train_pred[iii]))
    train_MAE.append(mean_absolute_error(Q_train[iii],Q_train_pred[iii]))
    
for iii,b in enumerate(y_test_in_pred_eln):
    Q_test_in_pred[iii,:] = empirical_model_curve_fit(np.abs(N_test_in[iii]),b[0],b[1],b[2])
    test_in_RMSE.append(root_mean_squared_error(Q_test_in[iii],Q_test_in_pred[iii]))
    test_in_MAE.append(mean_absolute_error(Q_test_in[iii],Q_test_in_pred[iii]))
    
for iii,b in enumerate(y_test_out_pred_eln):
    Q_test_out_pred[iii,:] = empirical_model_curve_fit(np.abs(N_test_out[iii]),b[0],b[1],b[2])
    test_out_RMSE.append(root_mean_squared_error(Q_test_out[iii],Q_test_out_pred[iii]))
    test_out_MAE.append(mean_absolute_error(Q_test_out[iii],Q_test_out_pred[iii]))

In [61]:

print(f'Mean MAE Training: {np.mean(train_MAE)*100:.2f}%')
print(f'Mean MAE High-DoD: {np.mean(test_in_MAE)*100:.2f}%')
print(f'Mean MAE Low-DoD: {np.mean(test_out_MAE)*100:.2f}%')

print(f'Mean RMSE Training: {np.mean(train_RMSE)*100:.2f}%')
print(f'Mean RMSE High-DoD: {np.mean(test_in_RMSE)*100:.2f}%')
print(f'Mean RMSE Low-DoD: {np.mean(test_out_RMSE)*100:.2f}%')


Mean MAE Training: 2.32%
Mean MAE High-DoD: 2.63%
Mean MAE Low-DoD: 13.62%
Mean RMSE Training: 3.08%
Mean RMSE High-DoD: 3.46%
Mean RMSE Low-DoD: 19.28%


# Neural network

Configure neural network structure

In [29]:
seed_ = 42
random.seed(seed_)
torch.manual_seed(seed_)
np.random.seed(seed_)
torch.use_deterministic_algorithms(True)

In [30]:
# Define the network
class network(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(network, self).__init__()
        # Define the layers
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))  # Input layer
        for i in range(1, len(hidden_sizes)):
            self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))  # Hidden layers
        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))  # Output layer
        
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        x = self.layers[-1](x)
        # x = torch.sigmoid(self.layers[-1](x))
        # x = torch.relu(self.layers[-1](x))
        return x
    
# Define the early stopping class
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Define the input size and output size
input_size = 10 # 10 pca features
output_size = 3 # 3 empirical model parameters

Prepare training dataset

In [31]:
train_dataset_CV = []
val_dataset_CV = []
for fold in range(10):
    train_cells = training_cells_CV[fold]
    val_cells = val_cells_CV[fold]

    X_train_fold = X_train_PCA[train_cells]
    X_val_fold = X_train_PCA[val_cells]
   
    Y_train_fold = y_train_scaled[train_cells]
    Y_val_fold = y_train_scaled[val_cells]
    # print(Y_val_fold)

    X_train_fold = torch.tensor(X_train_fold,dtype=torch.float32)
    X_val_fold = torch.tensor(X_val_fold,dtype=torch.float32)

    Y_train_fold = torch.tensor(Y_train_fold,dtype=torch.float32)
    Y_val_fold = torch.tensor(Y_val_fold,dtype=torch.float32)


    train_dataset = TensorDataset(X_train_fold,Y_train_fold)
    val_dataset = TensorDataset(X_val_fold,Y_val_fold)

    train_dataset_CV.append(train_dataset)
    val_dataset_CV.append(val_dataset)


Hyperparameter optimization

In [ ]:
best_model_path = 'Best_network/best_seq_model_V2.pth'  # File to save the best model
best_val_loss = float('inf')

# Hyperparameters to optimize: lr, # of neurons, # of layers
def objective(trial):
    global best_val_loss

    val_loss_CV = []
    model_state_dict = {}
    # Hyperparameter search space
    # lr = trial.suggest_float('lr', 5e-5, 5e-2,log=True)
    lr = trial.suggest_float('lr', 1e-4, 1e-1,log=True)
    num_neuron = trial.suggest_int('num_neuron',5,20)
    num_layer = trial.suggest_int('num_layer',2,5)
    batch_size = trial.suggest_int('batch_size',16,45)
    weight_decay = trial.suggest_float('weight_decay', 1e-8, 1e-6,log=True)

    hidden_sizes = [num_neuron]*num_layer # Mapping 

    # Define the starting epoch of checking early stopping and maximum number of epochs
    num_epochs = 1000
    warmup_epochs = 150

    for fold in range(10):
        # Load dataset for the current fold
        train_dataset = train_dataset_CV[fold]
        val_dataset = val_dataset_CV[fold]
        train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
        val_loader = DataLoader(val_dataset,batch_size=16,shuffle=False)
        
        # Initialize the model, loss function, optimizer, and early stopping
        model = network(input_size,hidden_sizes,output_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(),lr = lr,weight_decay=weight_decay)
        early_stopping = EarlyStopping(patience=25, min_delta=5e-5)

        # Training loop
        for epoch in range(num_epochs):
            # Training phase
            model.train()
            for batch in train_loader:
                inputs, labels = batch
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            # Validation phase
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    val_loss += loss.item()
            val_loss /= len(val_loader)

            # Check early stopping condition
            if epoch > warmup_epochs:
                early_stopping(val_loss)
                if early_stopping.early_stop:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
        
        val_loss_CV.append(val_loss)
        model_state_dict[fold] = model.state_dict()

    val_loss_overall = np.mean(val_loss_CV)
    print(f"Validation loss: {val_loss_CV}")
    # Check if the current validation loss is the best
    if val_loss_overall < best_val_loss:
        best_val_loss = val_loss_overall
        # Save the best model
        torch.save({'model_state':model_state_dict,
                    'lr':lr,
                    'num_neuron':num_neuron,
                    'num_layer':num_layer,
                    'batch_size':batch_size,
                    'weight_decay':weight_decay}, best_model_path)
    
    return val_loss_overall


sampler = optuna.samplers.TPESampler(seed=seed_)  # Make the sampler behave in a deterministic way.
study = optuna.create_study(sampler=sampler,direction='minimize')
# Run the optimization
study.optimize(objective, n_trials=50)

# Print the best parameters
print("Best hyperparameters:", study.best_params)

[I 2024-10-30 15:06:19,700] A new study created in memory with name: no-name-2ae6fe9d-174e-4070-824f-ec610d0a13ff


Early stopping at epoch 178
Early stopping at epoch 178
Early stopping at epoch 177
Early stopping at epoch 183
Early stopping at epoch 221
Early stopping at epoch 183
Early stopping at epoch 209
Early stopping at epoch 177
Early stopping at epoch 183


[I 2024-10-30 15:06:33,158] Trial 0 finished with value: 1.0977021753787994 and parameters: {'lr': 0.0013292918943162175, 'num_neuron': 20, 'num_layer': 4, 'batch_size': 33, 'weight_decay': 2.0513382630874456e-08}. Best is trial 0 with value: 1.0977021753787994.


Early stopping at epoch 178
Validation loss: [1.1024571657180786, 3.3808224201202393, 0.86934494972229, 1.0190231800079346, 0.7117137908935547, 1.1764955520629883, 0.2432132065296173, 0.4882822334766388, 1.4248842000961304, 0.5607850551605225]
Early stopping at epoch 764
Early stopping at epoch 348
Early stopping at epoch 293
Early stopping at epoch 177
Early stopping at epoch 181
Early stopping at epoch 177
Early stopping at epoch 264
Early stopping at epoch 324
Early stopping at epoch 230


[I 2024-10-30 15:06:55,311] Trial 1 finished with value: 0.8640991538763046 and parameters: {'lr': 0.00029375384576328325, 'num_neuron': 5, 'num_layer': 5, 'batch_size': 34, 'weight_decay': 2.6070247583707663e-07}. Best is trial 1 with value: 0.8640991538763046.


...

Early stopping at epoch 195
Validation loss: [0.4943748712539673, 1.4485098123550415, 1.2369672060012817, 0.4960533678531647, 0.5336349010467529, 0.8855083584785461, 0.27373796701431274, 0.4745265543460846, 1.0670452117919922, 0.4195774495601654]
Early stopping at epoch 199
Early stopping at epoch 182
Early stopping at epoch 210
Early stopping at epoch 204
Early stopping at epoch 190
Early stopping at epoch 222
Early stopping at epoch 177
Early stopping at epoch 177
Early stopping at epoch 191


[I 2024-10-30 15:22:21,386] Trial 48 finished with value: 1.2212288916110992 and parameters: {'lr': 0.015112885335772673, 'num_neuron': 19, 'num_layer': 3, 'batch_size': 27, 'weight_decay': 5.986265433897067e-08}. Best is trial 13 with value: 0.5523158699274063.


Early stopping at epoch 203
Validation loss: [0.6147568821907043, 5.079899311065674, 1.385925531387329, 1.2279316186904907, 0.6113917827606201, 0.8815726041793823, 0.5016729235649109, 0.6683135032653809, 0.9543002247810364, 0.28652453422546387]
Early stopping at epoch 177
Early stopping at epoch 177
Early stopping at epoch 225
Early stopping at epoch 178
Early stopping at epoch 183
Early stopping at epoch 178
Early stopping at epoch 180
Early stopping at epoch 177
Early stopping at epoch 177


[I 2024-10-30 15:22:33,815] Trial 49 finished with value: 0.8395147159695625 and parameters: {'lr': 0.0009023312763430668, 'num_neuron': 8, 'num_layer': 5, 'batch_size': 30, 'weight_decay': 2.5041332596460007e-08}. Best is trial 13 with value: 0.5523158699274063.


Early stopping at epoch 187
Validation loss: [0.6783896088600159, 0.8917067050933838, 1.8619134426116943, 0.9061877131462097, 0.6580027341842651, 1.1165767908096313, 0.329132080078125, 0.629479169845581, 1.1787093877792358, 0.14504952728748322]
Best hyperparameters: {'lr': 0.00011481963239014913, 'num_neuron': 16, 'num_layer': 3, 'batch_size': 23, 'weight_decay': 5.703654784743073e-08}


Predict empirical parameters using the neural network

In [50]:
best_model_dict = torch.load(best_model_path)
hidden_sizes = [best_model_dict['num_neuron']]*best_model_dict['num_layer']

# Test features as tensor
X_train_PCA_tensor = torch.tensor(X_train_PCA,dtype = torch.float32)
X_test_in_PCA_tensor = torch.tensor(X_test_in_PCA,dtype=torch.float32)
X_test_out_PCA_tensor = torch.tensor(X_test_out_PCA,dtype=torch.float32)

b_train_pred = []
b_test_in_pred = []
b_test_out_pred = []

Q_train_pred = []
Q_test_in_pred = []
Q_test_out_pred = []
for fold in range(10):
    # Load the model for the current fold
    model = network(input_size,hidden_sizes,output_size)
    model.load_state_dict(best_model_dict['model_state'][fold])

    # Evaluate the model with the completed training set and test sets
    model.eval()
    with torch.no_grad():
        Y_train_scaled_pred = model(X_train_PCA_tensor).numpy()
        Y_test_in_scaled_pred = model(X_test_in_PCA_tensor).numpy()
        Y_test_out_scaled_pred = model(X_test_out_PCA_tensor).numpy()

    # Convert predicted values back to the original scale
    b_train_pred_fold = np.exp(y_scaler.inverse_transform(Y_train_scaled_pred))
    b_test_in_pred_fold = np.exp(y_scaler.inverse_transform(Y_test_in_scaled_pred))
    b_test_out_pred_fold = np.exp(y_scaler.inverse_transform(Y_test_out_scaled_pred))

    
    Q_train_pred_fold = []
    for i in range(num_training_cells):
        b = b_train_pred_fold[i]
        Q_train_pred_fold.append(np.clip(empirical_model_curve_fit(N_train[i],*b),0,1))
    
    Q_test_in_pred_fold = []
    for i in range(num_test_in_cells):
        b = b_test_in_pred_fold[i]
        Q_test_in_pred_fold.append(np.clip(empirical_model_curve_fit(N_test_in[i],*b),0,1))

    Q_test_out_pred_fold = []
    for i in range(num_test_out_cells):
        b = b_test_out_pred_fold[i]
        Q_test_out_pred_fold.append(np.clip(empirical_model_curve_fit(N_test_out[i],*b),0,1))
    
    # Save the results from the current fold
    b_train_pred.append(np.array(b_train_pred_fold))
    b_test_in_pred.append(np.array(b_test_in_pred_fold))
    b_test_out_pred.append(np.array(b_test_out_pred_fold))

    Q_train_pred.append(np.array(Q_train_pred_fold))
    Q_test_in_pred.append(np.array(Q_test_in_pred_fold))
    Q_test_out_pred.append(np.array(Q_test_out_pred_fold))


# Write the results to files
with open('Empirical parameter results/seq_NN_V2/b_train_pred.pkl','wb') as f:
    pickle.dump(b_train_pred,f)

with open('Empirical parameter results/seq_NN_V2/b_test_in_pred.pkl','wb') as f:
    pickle.dump(b_test_in_pred,f)

with open('Empirical parameter results/seq_NN_V2/b_test_out_pred.pkl','wb') as f:
    pickle.dump(b_test_out_pred,f)

with open('Empirical parameter results/seq_NN_V2/Q_train_pred.pkl','wb') as f:
    pickle.dump(Q_train_pred,f)

with open('Empirical parameter results/seq_NN_V2/Q_test_in_pred.pkl','wb') as f:
    pickle.dump(Q_test_in_pred,f)

with open('Empirical parameter results/seq_NN_V2/Q_test_out_pred.pkl','wb') as f:
    pickle.dump(Q_test_out_pred,f)


Evaluate the prediction

In [ ]:
MAE_training = []
MAE_test_in = []
MAE_test_out = []

RMSE_training = []
RMSE_test_in = []
RMSE_test_out = []

for fold in range(10):
    MAE_training_fold = []
    MAE_test_in_fold = []
    MAE_test_out_fold = []

    RMSE_training_fold = []
    RMSE_test_in_fold = []
    RMSE_test_out_fold = []

    for i in range(num_training_cells):
        MAE_training_fold.append(mean_absolute_error(Q_train[i],Q_train_pred[fold][i]))
        RMSE_training_fold.append(root_mean_squared_error(Q_train[i],Q_train_pred[fold][i]))

    for i in range(num_test_in_cells):
        MAE_test_in_fold.append(mean_absolute_error(Q_test_in[i],Q_test_in_pred[fold][i]))
        RMSE_test_in_fold.append(root_mean_squared_error(Q_test_in[i],Q_test_in_pred[fold][i]))

    for i in range(num_test_out_cells):
        MAE_test_out_fold.append(mean_absolute_error(Q_test_out[i],Q_test_out_pred[fold][i]))
        RMSE_test_out_fold.append(root_mean_squared_error(Q_test_out[i],Q_test_out_pred[fold][i]))

    MAE_training.append(np.mean(MAE_training_fold)*100)
    MAE_test_in.append(np.mean(MAE_test_in_fold)*100)
    MAE_test_out.append(np.mean(MAE_test_out_fold)*100)

    RMSE_training.append(np.mean(RMSE_training_fold)*100)
    RMSE_test_in.append(np.mean(RMSE_test_in_fold)*100)
    RMSE_test_out.append(np.mean(RMSE_test_out_fold)*100)

result_dict = {'MAE_training':MAE_training,
                'MAE_test_in':MAE_test_in,
                'MAE_test_out':MAE_test_out,
                'RMSE_training':RMSE_training,
                'RMSE_test_in':RMSE_test_in,
                'RMSE_test_out':RMSE_test_out}

with open('Empirical_parameter_results/seq_NN_V2/result_dict.pkl','wb') as f:
    pickle.dump(result_dict,f)

mean_MAE_training = np.mean(MAE_training)
mean_MAE_test_in = np.mean(MAE_test_in)
mean_MAE_test_out = np.mean(MAE_test_out)

mean_RMSE_training = np.mean(RMSE_training)
mean_RMSE_test_in = np.mean(RMSE_test_in)
mean_RMSE_test_out = np.mean(RMSE_test_out)

print(f"Mean MAE training: {mean_MAE_training:.2f}")
print(f"Mean MAE test in: {mean_MAE_test_in:.2f}")
print(f"Mean MAE test out: {mean_MAE_test_out:.2f}")

print(f"Mean RMSE training: {mean_RMSE_training:.2f}")
print(f"Mean RMSE test in: {mean_RMSE_test_in:.2f}")
print(f"Mean RMSE test out: {mean_RMSE_test_out:.2f}")

Mean MAE training: 2.32
Mean MAE test in: 3.00
Mean MAE test out: 15.98
Mean RMSE training: 3.06
Mean RMSE test in: 4.05
Mean RMSE test out: 21.73


In [52]:
print(f"MAE training: {MAE_training}")
print(f"MAE test in: {MAE_test_in}")
print(f"MAE test out: {MAE_test_out}")

MAE training: [2.254962400218214, 2.1771970879089872, 2.669957631617043, 1.9455108934064875, 2.192069799484197, 1.9506564321878022, 2.4348079606098185, 2.6332592645473247, 2.5569290723312514, 2.366408315679782]
MAE test in: [3.3655910541803906, 2.6773868984177067, 3.145953080471129, 2.6052591802561973, 2.6991394989077775, 3.32692579762959, 2.684752414324961, 3.049348910615716, 3.010770956116634, 3.403895678032968]
MAE test out: [16.591792070878412, 15.54662660086555, 18.13388989257138, 17.477610188835197, 16.871421643123806, 16.904575493254928, 14.741220901542063, 17.55286152124765, 13.977633647569343, 12.044534820180578]
